# Build a compare view

Requires:
- Two VTK files for a given timestep
- A line segment

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import pyvista as pv
import os, subprocess, pickle
import pickle

In [ ]:
REPO_PATH = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode('utf-8').strip()
plt.style.use(f'{REPO_PATH}/misc/edwin.mplstyle')

In [ ]:
## Pyvista global settings
pv.set_plot_theme("document")
pv.set_jupyter_backend('static')
#pv.set_jupyter_backend('ipygany')
pv.global_theme.transparent_background = False
pv.global_theme.show_scalar_bar = True

In [ ]:
RASfile = os.path.join(REPO_PATH,"notebooks/HighDOC_NonZeroStart/2.0NonZeroInitCondition_7152.vtk")
LESfile = os.path.join(REPO_PATH,"notebooks/HighDOC_NonZeroStart/2.1NonZeroLESRun_22100222.vtk")

#vtkfile = os.path.join(REPO_PATH,"notebooks/LowDOC_reviewInititConc_17280.vtk")

RASgrid = pv.read(RASfile)
LESgrid = pv.read(LESfile)

In [ ]:
listOfSpecies = ['O2','DOC','NO3','NH4']
listOfAliveBiomass = ['XAR','XN','XDN']
listOfDeadBiomass  = ['EPS','XI']

In [39]:
colorbar_kw = dict(width=0.15, height=0.05, position_y=0.02, fmt='%.2E', label_font_size=14, n_labels=3)
addmesh_kw  = dict(log_scale=True, cmap='plasma', show_scalar_bar=True)
fileForPNG = "CompareMicrobialFields.png"

plotter = pv.Plotter(window_size=[800,850])

clim_arg = dict(xar = (1E-8,2E-1), xdn = (3E-5,5E-4), xn = (5E-6,1E-4))

slice_xar_ras = RASgrid.slice(normal='z')
slice_xar_ras.set_active_scalars('XAR',preference='point')
colorbar_kw['position_x'] = 0.05
plotter.add_mesh(slice_xar_ras, scalars='XAR', clim=clim_arg['xar'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)
#plotter.add_text("XAR",position=(233,650),font_size=12)

slice_xar_les = LESgrid.slice(normal='z')
slice_xar_les.translate((0, -0.21, 0), inplace=True)
slice_xar_les.set_active_scalars('XAR',preference='point')
colorbar_kw['position_x'] = 0.05
plotter.add_mesh(slice_xar_les, scalars='XAR', clim=clim_arg['xar'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)


slice_xdn_ras = RASgrid.slice(normal='z')
slice_xdn_ras.translate((0.16, 0, 0), inplace=True)
slice_xdn_ras.set_active_scalars('XDN',preference='point')
colorbar_kw['position_x'] = 0.34
plotter.add_mesh(slice_xdn_ras, scalars='XDN', clim=clim_arg['xdn'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)
#plotter.add_text("XDN",position=(700,650),font_size=12)

slice_xdn_les = LESgrid.slice(normal='z')
slice_xdn_les.translate((0.16, -0.21, 0), inplace=True)
slice_xdn_les.set_active_scalars('XDN',preference='point')
colorbar_kw['position_x'] = 0.34
plotter.add_mesh(slice_xdn_les, scalars='XDN', clim=clim_arg['xdn'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)


slice_xn_ras = RASgrid.slice(normal='z')
slice_xn_ras.translate((0.32, 0, 0), inplace=True)
slice_xn_ras.set_active_scalars('XN',preference='point')
colorbar_kw['position_x'] = 0.67
plotter.add_mesh(slice_xn_ras, scalars='XN', clim=clim_arg['xn'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)
#plotter.add_text("XN",position=(933,650),font_size=12)

slice_xn_les = LESgrid.slice(normal='z')
slice_xn_les.translate((0.32, -0.21, 0), inplace=True)
slice_xn_les.set_active_scalars('XN',preference='point')
colorbar_kw['position_x'] = 0.67
plotter.add_mesh(slice_xn_les, scalars='XN', clim=clim_arg['xn'],
                 scalar_bar_args=colorbar_kw, **addmesh_kw)

plotter.view_xy()
plotter.enable_parallel_projection()
plotter.camera.zoom(1.3)
plotter.screenshot(f"vtk_{fileForPNG}",transparent_background=True);
#os.system(f"xclip -se c -t image/png -i vtk_{fileForPNG}")
#plotter.show()